In [ ]:
! pip install chromadb
! pip install openai

In [2]:
import chromadb
from openai import OpenAI

In [3]:
openai_client = OpenAI(
    api_key='YOUR-OPENAI-API',
)

In [4]:
chroma_client = chromadb.Client()

In [6]:
# 이미 collection이 기존에 있다면
# chroma_client.delete_collection(name="fruit-db")

# 기존 collection에 계속 추가하고 싶다면
# collection = chroma_client.get_collection(name="fruite-db")

collection = chroma_client.create_collection(
        name="fruit-db",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [7]:
collection.add(
    documents=[
        "Orange is rich in vitamin C, which helps strengthen the immune system. With its tangy and sweet flavor, oranges are particularly popular in summer, and their juice is used in a variety of beverages.",
        "Grapes grow in clusters of many small berries. They are rich in vitamin K and antioxidants, which benefit cardiovascular health. Red grapes, in particular, contain resveratrol, which may help in anti-aging.",
        "Apple comes in various varieties, each with different tastes and textures. Apples are high in fiber, which supports digestive health, and contain a range of vitamins and minerals that aid in balanced nutrition.",
        "Banana is easily digestible and provides quick energy, making it a good choice for post-exercise recovery. Rich in potassium, bananas help regulate blood pressure and are commonly used in smoothies or as a snack.",
        "Kiwi has a small, fuzzy skin but is filled with a fresh, sweet juice inside. It is rich in vitamins C and E, which are effective for skin health and boosting the immune system.",
        "Strawberry is loved for its red color and fragrant taste. It is packed with antioxidants that can help reduce inflammation and is often used in desserts or salads due to its sweet flavor.",
        "Mango is a tropical fruit with a sweet and rich flavor. It is high in vitamins A and C, which support skin health and immunity, and is used in various dishes and beverages.",
        "Pineapple is a tropical fruit known for its tangy and sweet taste and contains the digestive enzyme bromelain. This enzyme aids digestion, and pineapple is often used in juices.",
        "Peach is a summer fruit known for its soft skin and sweet taste. Rich in vitamins A and C, it supports skin and immune system health and is used in a variety of desserts and dishes.",
        "Pear is a crisp and refreshing fruit popular in the summer. With its high water content and vitamin C, it is effective for quenching thirst and boosting immunity.",
        "Grapefruit is known for its tangy and slightly bitter taste, which can help stimulate metabolism. It is rich in vitamin C, benefiting immune health and skin care.",
        "Pomegranate contains many seeds, which are sweet and juicy. It is rich in antioxidants, which can benefit cardiovascular health and have potential anti-cancer effects.",
        "Mangosteen is a tropical fruit with a unique taste and aroma. It is high in vitamin C and antioxidants, which can support immune health and help with anti-aging.",
        "Blueberry is small but packed with antioxidants that support cognitive function and heart health. It is often used in smoothies and desserts.",
        "Mulberry is a dark purple fruit with a tangy and sweet taste. It is rich in vitamin C and dietary fiber, which can boost immunity and support digestive health.",
    ],
    ids=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15" ]

)

In [8]:
def retrieve_documents(query, collection):
    results = collection.query(query_texts=[query], n_results=1)
    most_similar_doc = results['documents']

    return most_similar_doc

In [9]:
def generate_response_RAG(query, relevant_document):
    prompt = f"Given the following document: '{relevant_document}', answer the query: '{query}'"

    response = openai_client.chat.completions.create(
    messages=[
        {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-3.5-turbo",
    )

    return response.choices[0].message.content

In [10]:
def generate_response_llm(query):
    prompt = f"Answer the query: '{query}'"

    response = openai_client.chat.completions.create(
    messages=[
        {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-3.5-turbo",
    )

    return response.choices[0].message.content

In [11]:
def rag_system(query, collection):
    relevant_document = retrieve_documents(query, collection)
    response = generate_response_RAG(query, relevant_document)
    return response

In [12]:
query = "Why is vitamin C important for health, and which fruits are high in it?"
response1 = generate_response_llm(query)
response2 = rag_system(query, collection)

print("[Query] \n", query)
print("=======================================================================")
print("[Response using LLM only] \n",response1)
print("=======================================================================")
print("[Response using RAG] \n",response2)

# other possible queries
# "What are the health benefits of oranges?"
# "How does eating grapes support cardiovascular health?"
# "What vitamins are found in kiwis and how do they benefit health?"
# "Describe the taste and texture of a peach."
# "How is a banana useful for post-exercise recovery?"
# "What makes a pomegranate unique among fruits?"
# "How are strawberries commonly used in recipes?"
# "What are some common ways to consume mangosteen?"
# "In what types of dishes can mango be used?"
# "Compare the health benefits of pears and apples."
# "How does the vitamin C content in grapefruits compare to that in oranges?"
# "What are the differences between blueberries and strawberries in terms of antioxidants?"
# "What enzyme does pineapple contain and what is its function?"
# "What are the key nutritional components of a kiwi?"
# "Describe the appearance and taste of mulberries."
# "Why is vitamin C important for health, and which fruits are high in it?"
# "How can antioxidants benefit your body, and which fruits are good sources?"
# "What are the benefits of dietary fiber found in fruits like pears and apples?"
# "What are some traditional dishes that use mangoes in tropical regions?"
# "How is grapefruit used in different cuisines around the world?"
# "In which regions are pomegranates considered a staple fruit?"

[Query] 
 Why is vitamin C important for health, and which fruits are high in it?
[Response using LLM only] 
 Vitamin C is important for maintaining overall health because it plays a crucial role in boosting the immune system, promoting healthy skin, wound healing, and aiding in the absorption of iron. It is also a powerful antioxidant that helps protect cells from damage caused by free radicals.

Some fruits that are high in vitamin C include:

1. Oranges
2. Kiwi
3. Strawberries
4. Guava
5. Pineapple
6. Mango
7. Papaya
8. Kiwi
9. Cantaloupe
10. Citrus fruits like lemons, limes, and grapefruits

Including these fruits in your diet regularly can help ensure you are getting enough vitamin C to support your overall health and well-being.
[Response using RAG] 
 Vitamin C is important for health because it helps strengthen the immune system. Oranges are high in vitamin C and are particularly popular in summer due to their tangy and sweet flavor.
